In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
orientation_path='/content/drive/MyDrive/Colab Notebooks/463/TrDataset/power-tr-train.tsv'
orientation_path

'/content/drive/MyDrive/Colab Notebooks/463/TrDataset/power-tr-train.tsv'

In [4]:
# Load the file
df = pd.read_csv(orientation_path, sep='\t')

# Print the first 2 rows
df.head(2)

,id,speaker,sex,text,text_en,label
0,tr18146,ca2031caa4032c51980160359953d507,M,"Yeni yasama döneminin ülkemiz için, milletimiz...","Mr. President, dear lawmakers, I salute you, a...",0
1,tr18147,4cee0addb3c69f6866869b180f90d45f,M,"Sayın Başkan, değerli milletvekilleri; bugün, ...","Mr. President, members of lawmakers, as I spea...",0


We check the proporiton of label to split the dataset in a stratified manner

In [5]:
df.keys()

Index(['id', 'speaker', 'sex', 'text', 'text_en', 'label'], dtype='object')

In [6]:
df["label"].value_counts(normalize=True)

,proportion
label,
1,0.513806
0,0.486194


In [7]:
print(df.isnull().sum())  # Check for missing values
print(df["label"].value_counts())  # Check label distribution

id         0
speaker    0
sex        0
text       0
text_en    0
label      0
dtype: int64
label
1    8932
0    8452
Name: count, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df["label"])
train_df, eval_df = train_test_split(train_df, test_size=0.111111, random_state=42, stratify=train_df["label"])

In [10]:
# Print the subset lengths/size and proportions to check if splitted successfully
print('Train:')
print(train_df.shape[0]/8)
print(train_df['label'].value_counts(normalize=True))

print('Eval:')
print(eval_df.shape[0]/1)
print(eval_df['label'].value_counts(normalize=True))

print('Test:')
print(test_df.shape[0]/1)
print(test_df['label'].value_counts(normalize=True))

Train:
1738.25
label
1    0.513807
0    0.486193
Name: proportion, dtype: float64
Eval:
1739.0
label
1    0.513514
0    0.486486
Name: proportion, dtype: float64
Test:
1739.0
label
1    0.514089
0    0.485911
Name: proportion, dtype: float64


In [11]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [12]:
#convert pandas dataFrame to Hugigng face dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset['text'][12]

"8 Marta giderken öncelikle tüm kadınların 8 Mart Dünya Kadınlar Günü’nü kutlayarak başlamak istiyorum. <p> Evet, bir kutlama günü gibi gözükse de aslında, bizim kalan diğer üç yüz altmış dört gün gibi mücadele ettiğimiz, kadın özgürlük çizgimizi büyütmeye çalıştığımız; haklarımızı, kazanılmış haklarımızı geri vermemek için mücadele etmek zorunda kaldığımız; yeni haklarımızı elde etmek için uğraştığımız bir günden bahsediyoruz. <p> 8 Mart, bizim açımızdan, tarihsel açıdan kadınların canlarıyla ödedikleri bedeller sonucu kazanılmış günlerden bir tanesi. Mücadeleyi büyütmekte kararlıyız. Kadın özgürlükçü çizgimizi büyütmekte ve erkek egemenliğinin olmadığı, patriyarkalliğin olmadığı, kapitalizmin olmadığı, kadın-erkek eşitliğinin sağlandığı bir dünyayı yaratmakta da kararlı olduğumuzu tüm kadınlar adına buradan bir kez daha haykırmak isteriz. Mücadelemiz devam edecek; bütün iktidarların engellemelerine rağmen, Türkiye'de olduğu gibi, kadınların tüm etkinlik ve eylemliliklerini engellemek

In [13]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [14]:
encoded_input_example = tokenizer("A fine but tarnished diamond is always preferable to a piece of glass, no matter how polished.")
encoded_input_example

{'input_ids': [0, 62, 5885, 1284, 2405, 93, 67175, 879, 15882, 83, 11343, 12601, 2886, 47, 10, 63847, 111, 47589, 4, 110, 26866, 3642, 13492, 5252, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.decode(encoded_input_example["input_ids"])

'<s> A fine but tarnished diamond is always preferable to a piece of glass, no matter how polished.</s>'

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text_en"], padding="max_length", truncation=True, max_length=512)

In [17]:
unshuffled_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset =unshuffled_train_dataset.shuffle(seed=42)

unshuffled_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset =unshuffled_eval_dataset.shuffle(seed=42)

Map:   0%|          | 0/13906 [00:00<?, ? examples/s]

Map:   0%|          | 0/1739 [00:00<?, ? examples/s]

In [18]:
tokenized_train_dataset.column_names

['id',
 'speaker',
 'sex',
 'text',
 'text_en',
 'label',
 '__index_level_0__',
 'input_ids',
 'attention_mask']

Start Pre-Training using Hugging-Face Transformers

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments
batch_size = 32
grad_acc_steps = 4
num_epochs = 3
training_args = TrainingArguments(
    output_dir="./test_trainer",
    eval_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    logging_steps=20,
    eval_steps=20,
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=grad_acc_steps,
    num_train_epochs=num_epochs,
    warmup_ratio=0.1,
    #warmup_steps=int(0.1* 12910/(batch_size*grad_acc_steps/num_epochs)),
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none",  # Disable reporting to external platforms
)


# Evaluate

In [21]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00


In [22]:
import numpy as np
import evaluate

In [23]:
metric = evaluate.load("accuracy")

In [24]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_score(labels, predictions, average="binary")
    return {"accuracy": accuracy, "f1": f1}

# Trainer

In [25]:
from transformers import Trainer
from transformers import EarlyStoppingCallback
from transformers import TrainerCallback

In [26]:
class LearningRateAndEvalLogger(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Log evaluation metrics
        print(f"Step {state.global_step}: Evaluation Metrics: {metrics}")

        # Log the current learning rate
        if state.global_step > 0 and kwargs.get("optimizer"):
            lr = kwargs["optimizer"].param_groups[0]["lr"]
            print(f"Step {state.global_step}: Learning Rate = {lr:.8f}")


In [27]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_eval_dataset,
   compute_metrics=compute_metrics,
   callbacks=[LearningRateAndEvalLogger(), EarlyStoppingCallback(early_stopping_patience=3)],
)

In [28]:
#Example
example = tokenized_eval_dataset[0]
print("Example:", example)

Example: {'id': 'tr21974', 'speaker': '9fc862a54989ebca9fce2d6dd0add4bf', 'sex': 'M', 'text': "Sayın Başkan, değerli milletvekilleri; 9 ve 10’uncu maddelerde 1163 sayılı Kooperatifler Kanunu üzerine iki düzenleme var, ikisi aynı mahiyette. <p> İlk hâliyle komisyona geldiğinde “kamu tüzel kişiliği” ibaresi vardı “üniversiteler” yerine. Yani, bir kamu tüzel kişisi, bir kooperatif ortağı ise orada kooperatif ana kuralı olan her ortak tek oy kuralı yerine, kamu tüzel kişiliğine payı kadar, sermayesi kadar oy imkânı veriyordu. Şimdi, bu, çok sakıncalı tabii. Yani, 1163’ün, Kooperatifler Yasası’nın esasına aykırı, âdeta bir virüs gibi bunun içine giren, bir yol açan… Eğer bu girerse bundan sonra hangi kooperatiflere, nereye, ne girer belli olmaz. <p> Şimdi, bu, daraltılmış gözüküyor alt komisyonda verilen bir önergeyle. Doğru, en azından daraltılması. Fakat bunun da başka sakıncaları var. Yani, daraltıldı, üniversiteler kondu yani üniversitelerde… Ki eskiden biliyorsunuz vakıflar vardı, vakı

In [29]:
eval_results = trainer.evaluate()
#print(eval_results)  # Includes the accuracy metric

Step 0: Evaluation Metrics: {'eval_loss': 0.6937976479530334, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.5135135135135135, 'eval_f1': 0.6785714285714286, 'eval_runtime': 7.9467, 'eval_samples_per_second': 218.833, 'eval_steps_per_second': 13.716}


In [30]:
trainer.train()

Step,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
20,2.784100,0.691778,0.003700,0.513514,0.678571
40,2.696300,0.608231,0.003700,0.697527,0.692757
60,2.185100,0.508919,0.003700,0.763082,0.761298
80,1.902300,0.485368,0.003700,0.798160,0.806824
100,1.797100,0.440193,0.003700,0.811961,0.830306
120,1.738100,0.375058,0.003700,0.838988,0.846323
140,1.662700,0.528025,0.003700,0.783784,0.817829
160,1.575800,0.378297,0.003700,0.838413,0.851401
180,1.383800,0.338803,0.003700,0.853939,0.855846
200,1.363100,0.353846,0.003700,0.857964,0.853586


Step 20: Evaluation Metrics: {'eval_loss': 0.6917777061462402, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.5135135135135135, 'eval_f1': 0.6785714285714286, 'eval_runtime': 6.9853, 'eval_samples_per_second': 248.952, 'eval_steps_per_second': 15.604, 'epoch': 0.1839080459770115}
Step 20: Learning Rate = 0.00001212
Step 40: Evaluation Metrics: {'eval_loss': 0.6082308292388916, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.6975273145485912, 'eval_f1': 0.6927570093457944, 'eval_runtime': 7.2265, 'eval_samples_per_second': 240.642, 'eval_steps_per_second': 15.083, 'epoch': 0.367816091954023}
Step 40: Learning Rate = 0.00001952
Step 60: Evaluation Metrics: {'eval_loss': 0.5089191794395447, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.7630822311673375, 'eval_f1': 0.761297798377752, 'eval_runtime': 7.1021, 'eval_samples_per_second': 244.856, 'eval_steps_per_second': 15.347, 'epoch': 0.5517241379310345}
Step 60: Learning Rate = 0.00001835
Step 80: Evaluatio

Could not locate the best model at ./test_trainer/checkpoint-320/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=324, training_loss=1.6376416388853097, metrics={'train_runtime': 754.2343, 'train_samples_per_second': 55.312, 'train_steps_per_second': 0.43, 'total_flos': 1.0945419902976e+16, 'train_loss': 1.6376416388853097, 'epoch': 2.9931034482758623})

In [31]:
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1739 [00:00<?, ? examples/s]

In [32]:
print(f'Train Dataset Column Names:{train_dataset.column_names}')
print(f'Tokenized Train Dataset Column Names:{tokenized_train_dataset.column_names}')
example = tokenized_train_dataset[0]
print(f'first element of tokenized dateset{example}')


Train Dataset Column Names:['id', 'speaker', 'sex', 'text', 'text_en', 'label', '__index_level_0__']
Tokenized Train Dataset Column Names:['id', 'speaker', 'sex', 'text', 'text_en', 'label', '__index_level_0__', 'input_ids', 'attention_mask']
first element of tokenized dateset{'id': 'tr22454', 'speaker': '8b990f6eb6f268ccf94f9b3e67a0aa14', 'sex': 'M', 'text': "Değerli milletvekilleri, emniyet mensubu vazife malulleri ile şehit gazi ve malul personel eş ve çocuğu olanların, emniyet teşkilatı mensubu şehit ve gazi aileleri ile yakınlarının kurdukları şehit ve gazilere yönelik yardımlaşma derneklerine ve vakıflarına üye olabilmeleri hakkındaki kanun teklifi üzerinde söz almış bulunuyorum. Hepinizi saygıyla selamlıyorum. <p> Değerli milletvekilleri, bütün demokratik hukuk devletlerinde polis, adaletin başladığı kapı, adalete gireceğimiz kapı. Bu nedenle, eğer, gerçekten ülkemizde demokrasi olsun, hukuk olsun istiyorsak bir kere, gerçekten, polisin eğitim düzeyini, ekonomik ve sosyal alanda

In [33]:
print(tokenized_train_dataset['label'][:10])  # Check the first 10 labels

# Ensure labels are integers
if not all(isinstance(label, int) for label in tokenized_train_dataset['label']):
    print("Converting labels to integers...")
    train_dataset = train_dataset.map(lambda x: {"label": int(x["label"])})
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)


[1, 0, 0, 0, 0, 0, 1, 1, 1, 0]


In [34]:
test_results = trainer.evaluate(tokenized_test_dataset)
print("Test Set Results:", test_results)

Step 324: Evaluation Metrics: {'eval_loss': 0.34821832180023193, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.851063829787234, 'eval_f1': 0.8550643536653609, 'eval_runtime': 7.0364, 'eval_samples_per_second': 247.143, 'eval_steps_per_second': 15.491, 'epoch': 2.9931034482758623}
Step 324: Learning Rate = 0.00000027
Test Set Results: {'eval_loss': 0.34821832180023193, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.851063829787234, 'eval_f1': 0.8550643536653609, 'eval_runtime': 7.0364, 'eval_samples_per_second': 247.143, 'eval_steps_per_second': 15.491, 'epoch': 2.9931034482758623}


In [35]:
# Save the model and tokenizer
model.save_pretrained("./task2_xlm-roberta-base_fine-tune")
tokenizer.save_pretrained("./task2_xlm-roberta-base_fine-tune")

('./task2_xlm-roberta-base_fine-tune/tokenizer_config.json',
 './task2_xlm-roberta-base_fine-tune/special_tokens_map.json',
 './task2_xlm-roberta-base_fine-tune/sentencepiece.bpe.model',
 './task2_xlm-roberta-base_fine-tune/added_tokens.json',
 './task2_xlm-roberta-base_fine-tune/tokenizer.json')